In [2]:
import numpy as np

In [3]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P, Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)

                    for k in range(K):
                        e = e + (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        # 0.001: local minimum
        if e < 0.001:
            break

    return P, Q.T

In [6]:
with open('user_movie_matrix.npy', 'rb') as f:
    R = np.load(f)

In [8]:
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 3

In [9]:
P = np.random.rand(N, K)
Q = np.random.rand(M, K)

In [10]:
nP, nQ = matrix_factorization(R, P, Q, K)

In [11]:
nR = np.dot(nP, nQ.T)

In [12]:
print(nR)

[[2.74969538 4.04727203 0.94122593 ... 6.29336143 4.75272138 4.14276822]
 [2.68712563 4.11330793 0.93972249 ... 6.31125901 4.81335932 4.19511279]
 [2.43731202 4.00844429 0.98952095 ... 6.17708188 5.08864072 3.92138557]
 [2.61246974 3.69454961 1.07343362 ... 6.15493412 5.18158221 3.52280757]]


In [15]:
with open('collab_filter_random_features.npy', 'wb') as f:
    np.save(f, nR)